# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']


J. Liu  ->  J. Liu  |  ['J. Liu']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 83 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.02157


extracting tarball to tmp_2409.02157... done.
Retrieving document from  https://arxiv.org/e-print/2409.02194


extracting tarball to tmp_2409.02194... done.
Retrieving document from  https://arxiv.org/e-print/2409.02286


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2676/1211882699.py:51: LatexWarning: 2409.02194 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2409.02405/mnras_template.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2409.02734... done.
Retrieving document from  https://arxiv.org/e-print/2409.02783


extracting tarball to tmp_2409.02783...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2676/1211882699.py:51: LatexWarning: 2409.02783 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02157-b31b1b.svg)](https://arxiv.org/abs/2409.02157) | **An Earth-Mass Planet and a Brown Dwarf in Orbit Around a White Dwarf**  |
|| K. Zhang, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2024-09-05*|
|*Comments*| *Accepted. 25 pages, 7 figures, 4 tables*|
|**Abstract**|            Terrestrial planets born beyond 1-3 AU have been theorized to avoid being engulfed during the red-giant phases of their host stars. Nevertheless, only a few gas-giant planets have been observed around white dwarfs (WDs) -- the end product left behind by a red giant. Here we report on evidence that the lens system that produced the microlensing event KMT-2020-BLG-0414 is composed of a WD orbited by an Earth-mass planet and a brown dwarf (BD) companion, as shown by the non-detection of the lens flux using Keck Adaptive Optics (AO). From microlensing orbital motion constraints, we determine the planet to be a $1.9\pm0.2$ Earth-mass ($M_\oplus$) planet at a physical separation of $2.1\pm0.2$ au from the WD during the event. By considering the system evolutionary history, we determine the BD companion to have a projected separation of 22 au from the WD, and reject an alternative model that places the BD at 0.2 au. Given planetary orbital expansion during the final evolutionary stages of the host star, this Earth-mass planet may have existed in an initial orbit close to 1 au, thereby offering a glimpse into the possible survival of planet Earth in the distant future.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02286-b31b1b.svg)](https://arxiv.org/abs/2409.02286) | **The Wanderer: Charting WASP-77A b's Formation and Migration Using a System-Wide Inventory of Carbon and Oxygen Abundances**  |
|| D. R. Coria, N. Hejazi, <mark>I. J. M. Crossfield</mark>, M. Rhem |
|*Appeared on*| *2024-09-05*|
|*Comments*| *21 pages, 4 figures, 3 tables Accepted for publication in ApJ (8/14/2024)*|
|**Abstract**|            The elemental and isotopic abundances of volatiles like carbon, oxygen, and nitrogen may trace a planet's formation location relative to H$_2$O, CO$_2$, CO, NH$_3$, and N$_2$ "snowlines", or the distance from the star at which these volatile elements sublimate. By comparing the C/O and $^{12}$C/$^{13}$C ratios measured in giant exoplanet atmospheres to complementary measurements of their host stars, we can determine whether the planet inherited stellar abundances from formation inside the volatile snowlines, or non-stellar C/O and $^{13}$C enrichment characteristic of formation beyond the snowlines. To date, there are still only a handful of exoplanet systems where we can make a direct comparison of elemental and isotopic CNO abundances between an exoplanet and its host star. Here, we present a $^{12}$C/$^{13}$C abundance analysis for host star WASP-77A (whose hot Jupiter's $^{12}$C/$^{13}$C abundance was recently measured). We use MARCS stellar atmosphere models and the radiative transfer code TurboSpectrum to generate synthetic stellar spectra for isotopic abundance calculations. We find a $^{12}$C/$^{13}$C ratio of $51\pm 6$ for WASP-77A, which is sub-solar ($\sim 91$) but may still indicate $^{13}$C-enrichment in its companion planet WASP-77A b ($^{12}$C/$^{13}$C = 26 $\pm$ 16, previously reported). Together with the inventory of carbon and oxygen abundances in both the host and companion planet, these chemical constraints point to WASP-77A b's formation beyond the H$_2$O and CO$_2$ snowlines and provide chemical evidence for the planet's migration to its current location $\sim$0.024 AU from its host star.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02405-b31b1b.svg)](https://arxiv.org/abs/2409.02405) | **Rapid Automatic Multiple Moving Objects Detection Method Based on Feature Extraction from Images with Non-sidereal Tracking**  |
|| L. Wang, et al. -- incl., <mark>X. Zhang</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2024-09-05*|
|*Comments*| **|
|**Abstract**|            Optically observing and monitoring moving objects, both natural and artificial, is important to human space security. Non-sidereal tracking can improve the system's limiting magnitude for moving objects, which benefits the surveillance. However, images with non-sidereal tracking include complex background, as well as objects with different brightness and moving mode, posing a significant challenge for accurate multi-object detection in such images, especially in wide field of view (WFOV) telescope images. To achieve a higher detection precision in a higher speed, we proposed a novel object detection method, which combines the source feature extraction and the neural network. First, our method extracts object features from optical images such as centroid, shape, and flux. Then it conducts a naive labeling based on those features to distinguish moving objects from stars. After balancing the labeled data, we employ it to train a neural network aimed at creating a classification model for point-like and streak-like objects. Ultimately, based on the neural network model's classification outcomes, moving objects whose motion modes consistent with the tracked objects are detected via track association, while objects with different motion modes are detected using morphological statistics. The validation, based on the space objects images captured in target tracking mode with the 1-meter telescope at Nanshan, Xinjiang Astronomical Observatory, demonstrates that our method achieves 94.72% detection accuracy with merely 5.02% false alarm rate, and a processing time of 0.66s per frame. Consequently, our method can rapidly and accurately detect objects with different motion modes from wide-field images with non-sidereal tracking.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02734-b31b1b.svg)](https://arxiv.org/abs/2409.02734) | **Reply to Comment on "A slightly oblate dark matter halo revealed by a retrograde precessing Galactic disk warp"**  |
|| Y. Huang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-09-05*|
|*Comments*| *4 pages, 2 figures, 1 table, in response to Dehnen et al. (arXiv:2407.06341)*|
|**Abstract**|            In this reply, we present a comprehensive analysis addressing the concerns raised by Dehnen et al. (2024) regarding our recent measurement of the disk warp precession using the `motion-picture' method (Huang et al. 2024). We carefully examine the impact of ignoring the twist of the disk warp and the so-called $R$-$\tau$ correlation on the estimation of the precession rate. The results indicate that the effect is minor and does not exceed the systematic and statistical uncertainties. Using N-body+SPH simulation data, we confirm that the `motion-picture' technique is effective in measuring retrograde precession of disk warp in stellar populations younger than 170 Myr, similar to classical Cepheids. Therefore, the overall conclusions of Huang et al. (2024) remain robust.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02194-b31b1b.svg)](https://arxiv.org/abs/2409.02194) | **Galaxy clustering in a new implementation of the GAEA semi-analytical galaxy formation model**  |
|| F. Fontanot, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2024-09-05*|
|*Comments*| *10 Pages, 5 figures, 2 tables, 1 appendix - A&A submitted*|
|**Abstract**|            We present results from the latest version of the GAEA model of galaxy formation coupled with merger trees extracted from the P-Millennium Simulation (PMS), which provides a better mass resolution, a larger volume and assumes cosmological parameters consistent with latest results from the Planck mission. The model includes, at the same time, a treatment for the partition of cold gas into atomic and molecular (H$_2$) components; a better treatment for environmental processes acting on satellite galaxies; an updated modelling of cold gas accretion on Super-Massive Black Hole and relative AGN feedback on the host galaxy. We compare GAEA predictions based on the PMS, with model realizations based on other simulations in the Millennium Suite at different resolution, showing that the new model provides a remarkable consistency in the statistical properties of galaxy populations. We interpret this as due to the interplay between AGN feedback and H$_2$-based SFR (both acting as regulators of the cold gas content). We then compare model predictions with available data for the galaxy 2-point correlation function (2pCF) in the redshift range 0<z<3. We show that GAEA runs are able to correctly recover the main dependencies of the 2pCF as a function of stellar mass (M$_\star$), star formation activity, HI-content and redshift for M$_\star$ < 10$^{11}$ M$_\odot$ galaxies. Our model correctly captures both the distribution of galaxy populations in the Large Scale Structure and the interplay between the main physical processes regulating their baryonic content, both for central and satellite galaxies. At larger stellar masses GAEA underpredicts the 2pCF amplitude, suggesting that model massive galaxies live in less massive dark matter haloes. The model predicts a rather small redshift evolution of the clustering amplitude up to z$\sim$3, consistent with available observational evidence.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.02783-b31b1b.svg)](https://arxiv.org/abs/2409.02783) | **Euclid preparation: Determining the weak lensing mass accuracy and precision for galaxy clusters**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-09-05*|
|*Comments*| **|
|**Abstract**|            We investigate the level of accuracy and precision of cluster weak-lensing (WL) masses measured with the \Euclid data processing pipeline. We use the DEMNUni-Cov $N$-body simulations to assess how well the WL mass probes the true halo mass, and, then, how well WL masses can be recovered in the presence of measurement uncertainties. We consider different halo mass density models, priors, and mass point estimates. WL mass differs from true mass due to, e.g., the intrinsic ellipticity of sources, correlated or uncorrelated matter and large-scale structure, halo triaxiality and orientation, and merging or irregular morphology. In an ideal scenario without observational or measurement errors, the maximum likelihood estimator is the most accurate, with WL masses biased low by $\langle b_M \rangle = -14.6 \pm 1.7 \, \%$ on average over the full range $M_\text{200c} > 5 \times 10^{13} \, M_\odot$ and $z < 1$. Due to the stabilising effect of the prior, the biweight, mean, and median estimates are more precise. The scatter decreases with increasing mass and informative priors significantly reduce the scatter. Halo mass density profiles with a truncation provide better fits to the lensing signal, while the accuracy and precision are not significantly affected. We further investigate the impact of additional sources of systematic uncertainty on the WL mass, namely the impact of photometric redshift uncertainties and source selection, the expected performance of \Euclid cluster detection algorithms, and the presence of masks. Taken in isolation, we find that the largest effect is induced by non-conservative source selection. This effect can be mostly removed with a robust selection. As a final \Euclid-like test, we combine systematic effects in a realistic observational setting and find results similar to the ideal case, $\langle b_M \rangle = - 15.5 \pm 2.4 \, \%$, under a robust selection.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

171  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
